# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Inference Pipeline</span>


## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [13]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

2025-11-21 17:07:19,389 INFO: Closing external client and cleaning up certificates.
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
2025-11-21 17:07:19,413 INFO: Connection closed.
2025-11-21 17:07:19,427 INFO: Initializing external client
2025-11-21 17:07:19,427 INFO: Base URL: https://10.87.45.81:28181
2025-11-21 17:07:19,926 INFO: Python Engine initialized.

Logged in to project, explore it here https://10.87.45.81:28181/p/120


## <span style="color:#ff5f27;"> ⚙️ Feature Group Retrieval</span>
Let's retrieve a feature group in order to get cc_num values.

In [14]:
# Retrieve the 'transactions_fraud_online_fg' feature group
trans_fg = fs.get_feature_group(
    'transactions_fraud_online_fg',
    version=1,
)

In [15]:
# Retrieve the first 5 unique credit card numbers (cc_nums)
cc_nums = trans_fg.select('cc_num').show(5).cc_num.values

# Display the obtained cc_nums
cc_nums

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.97s) 


array([4896635955418431, 4837978576778318, 4003885636606306,
       4085049862790828, 4130716471597621])

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [16]:
# Get the Model Registry
mr = project.get_model_registry()

## <span style='color:#ff5f27'>🚀 Fetch Deployment</span>

In [17]:
# Access the Model Serving
ms = project.get_model_serving()

# Specify the deployment name
deployment_name = "fraudonlinemodeldeployment"

# Get the deployment with the specified name
deployment = ms.get_deployment(deployment_name)

# Start the deployment and wait for it to be in a running state for up to 300 seconds
deployment.start(await_running=300)

  0%|          | 0/5 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


## <span style='color:#ff5f27'>🔮 Predicting using deployment</span>


Finally you can start making predictions with your model!

Send inference requests to the deployed model as follows:

In [18]:
# Get the first credit card number
cc_num = cc_nums[0]
cc_num

4896635955418431

In [19]:
# Make a prediction
deployment.predict(
    inputs=[int(cc_num)],
)

{'predictions': [0]}

In [20]:
# Predict for several cc_nums
predictions = [
    deployment.predict(inputs=[int(cc_num)])['predictions'] 
    for cc_num
    in cc_nums
]
predictions

[[0], [0], [0], [0], [0]]

## <span style='color:#ff5f27'>🔮 Read Prediction Log</span>

In [21]:
retrieved_model = mr.get_model(
    name="xgboost_fraud_online_model",
    version=1,
)
feature_view = retrieved_model.get_feature_view()

2025-11-21 17:08:04,840 INFO: Initializing for batch retrieval of feature vectors


In [ ]:
# Stop the job materialization schedule and materialize log manually
feature_view.pause_logging()
feature_view.materialize_log(wait=True)

Launching job: transactions_fraud_online_fv_1_log_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://10.87.45.81:28181/p/120/jobs/named/transactions_fraud_online_fv_1_log_1_offline_fg_materialization/executions
2025-11-21 17:08:14,423 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-21 17:08:17,509 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


In [ ]:
# Read untransformed log
feature_view.read_log().head(3)

In [ ]:
# Read transformed log
feature_view.read_log(transformed=True).head(3)

### Stop Deployment
To stop the deployment you simply run:

In [ ]:
# Stop the deployment
deployment.stop(await_stopped=180)

## <span style="color:#ff5f27;">👾 StreamLit App</span>


If you want to see interactive dashboards - use a **StreamLit App**.

Type the next commands in terminal to run a Streamlit App:

`python -m streamlit run streamlit_app.py`

---
## <span style="color:#ff5f27;">⏭️ **Next:** Part 04: Model Monitoring</span>

In the following notebook you will monitor your model using the logs that have been written.